In [16]:
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter
from numpy import dot
from numpy.linalg import norm
#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt

# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0826/' #있다면 추가 해주기 (상대경로)
line_color_list = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'] 
# Global Variable Setting

# Nicname = '5CAC8432AE4EC54B'
# Huzzi = '44754E5A077F32B4'
# oizi = '2DDB3706DE4F7B45'
kyounghwan = '3963650B38CAF112'
hyeryeong = '4CCF4C0B0F953026'
pyeongju = '8A57AE95DD4629A7'
joongho = '9E7F4CB4F4A6A6D4'
yujin = 'F9888CC2D3053565'

PATIENT_UID = kyounghwan # 감염자 UID
TARGET_UID =  hyeryeong # 조사 대상자 UID

# Map Setting
Raw_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
Filter_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
mass_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
compare_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

#Call CSV file
def load_GPS_set(UID, scnario): # Data load
    try:
        if scnario == 1:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set
        elif scnario == 2:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set   
    except FileNotFoundError as e:
        pass

def load_ground_truth(UID):
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
        # data_set['TIME'] = data_set['TIME STAMP'].dt.strftime('%Y-%m-%d %H:%M') # 초단위 자르기
        # data_set['STAMP_TIME'] = pd.to_datetime(data_set['TIME']) # stamp_time datetime 형식으로 변경
        re_data_set = data_set[["TIME STAMP","4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
        return re_data_set
    
    except FileNotFoundError as e:
        pass

################### RAW DATA FUNC ######################
def extract_raw_data(UID, scnario):
    Dataframe = load_GPS_set(UID, scnario)
    marker_list = []
    for count in range(len(Dataframe.index)):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list
    
################### ABOUT MAP DROWING FUNC #####################
def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[1:3])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)

def draw_index_map(map, marker_list):
    count = len(marker_list)
    for marker_point in marker_list:
        folium.Marker(marker_point[1:3], icon=DivIcon(icon_size=(10,3),icon_anchor=(7,20),
        html='<div style="font-size: 10pt; color : black">'+str(count)+'</div>',
        )).add_to(map)
        count = count - 1

def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[1:3], radius=draw_radius, color= draw_color).add_to(map)
        
def draw_arrow_map(map, start_list, end_list, line_color, dot_color):
    for start_point in start_list:
        for end_point in end_list:
            if(start_point[0]==end_point[0]):
                try:
                    rot_x = end_point[1] - start_point[1]
                    rot_y = end_point[2] - start_point[2]
                    rot = math.acos(haversine([start_point[1],0],[end_point[1],0])/haversine(start_point[1:3],end_point[1:3]))
                    # folium.CircleMarker(location=end_point[1:3], radius=1, color= dot_color).add_to(map)
                    folium.PolyLine(locations=[start_point[1:3],end_point[1:3]],tooltip='Polyline',color=line_color).add_to(map)
                    folium.CircleMarker(location=start_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.RegularPolygonMarker(end_point[1:3], fill_color='blue', number_of_sides=3, radius=3, rotation=rot).add_to(map)
                except IndexError :
                    break
                except ZeroDivisionError:
                    break
                
def extractTime(UID, scnario):
    DataFrame = load_GPS_set(UID, scnario)
    timeStamp = np.unique(DataFrame["TIME STAMP"])
    return timeStamp

kyounghwan_raw = extract_raw_data('3963650B38CAF112', 1)
hyeryeong_raw = extract_raw_data('4CCF4C0B0F953026', 1)
pyeongju_raw = extract_raw_data('8A57AE95DD4629A7', 1)
joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4', 1)
yujin_raw = extract_raw_data('F9888CC2D3053565', 1)
draw_Polyline_map(Raw_map,yujin_raw,"red")
draw_Polyline_map(Raw_map,pyeongju_raw,"blue")
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter
from numpy import dot
from numpy.linalg import norm
#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt

# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0826/' #있다면 추가 해주기 (상대경로)
line_color_list = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'] 
# Global Variable Setting

# Nicname = '5CAC8432AE4EC54B'
# Huzzi = '44754E5A077F32B4'
# oizi = '2DDB3706DE4F7B45'
kyounghwan = '3963650B38CAF112'
hyeryeong = '4CCF4C0B0F953026'
pyeongju = '8A57AE95DD4629A7'
joongho = '9E7F4CB4F4A6A6D4'
yujin = 'F9888CC2D3053565'

PATIENT_UID = kyounghwan # 감염자 UID
TARGET_UID =  hyeryeong # 조사 대상자 UID

# Map Setting
Raw_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
Filter_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
mass_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
compare_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

#Call CSV file
def load_GPS_set(UID, scnario): # Data load
    try:
        if scnario == 1:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set
        elif scnario == 2:
            data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set   
    except FileNotFoundError as e:
        pass

def load_ground_truth(UID):
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
        # data_set['TIME'] = data_set['TIME STAMP'].dt.strftime('%Y-%m-%d %H:%M') # 초단위 자르기
        # data_set['STAMP_TIME'] = pd.to_datetime(data_set['TIME']) # stamp_time datetime 형식으로 변경
        re_data_set = data_set[["TIME STAMP","4CCF4C0B0F953026","3963650B38CAF112","8A57AE95DD4629A7","9E7F4CB4F4A6A6D4","F9888CC2D3053565"]]
        return re_data_set
    
    except FileNotFoundError as e:
        pass

################### RAW DATA FUNC ######################
def extract_raw_data(UID, scnario):
    Dataframe = load_GPS_set(UID, scnario)
    marker_list = []
    for count in range(len(Dataframe.index)):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list
    
################### ABOUT MAP DROWING FUNC #####################
def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[1:3])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)

def draw_index_map(map, marker_list):
    count = len(marker_list)
    for marker_point in marker_list:
        folium.Marker(marker_point[1:3], icon=DivIcon(icon_size=(10,3),icon_anchor=(7,20),
        html='<div style="font-size: 10pt; color : black">'+str(count)+'</div>',
        )).add_to(map)
        count = count - 1

def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[1:3], radius=draw_radius, color= draw_color).add_to(map)
        
def draw_arrow_map(map, start_list, end_list, line_color, dot_color):
    for start_point in start_list:
        for end_point in end_list:
            if(start_point[0]==end_point[0]):
                try:
                    rot_x = end_point[1] - start_point[1]
                    rot_y = end_point[2] - start_point[2]
                    rot = math.acos(haversine([start_point[1],0],[end_point[1],0])/haversine(start_point[1:3],end_point[1:3]))
                    # folium.CircleMarker(location=end_point[1:3], radius=1, color= dot_color).add_to(map)
                    folium.PolyLine(locations=[start_point[1:3],end_point[1:3]],tooltip='Polyline',color=line_color).add_to(map)
                    folium.CircleMarker(location=start_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.RegularPolygonMarker(end_point[1:3], fill_color='blue', number_of_sides=3, radius=3, rotation=rot).add_to(map)
                except IndexError :
                    break
                except ZeroDivisionError:
                    break
                
def extractTime(UID, scnario):
    DataFrame = load_GPS_set(UID, scnario)
    timeStamp = np.unique(DataFrame["TIME STAMP"])
    return timeStamp

kyounghwan_raw = extract_raw_data('3963650B38CAF112', 1)
hyeryeong_raw = extract_raw_data('4CCF4C0B0F953026', 1)
pyeongju_raw = extract_raw_data('8A57AE95DD4629A7', 1)
joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4', 1)
yujin_raw = extract_raw_data('F9888CC2D3053565', 1)
draw_Polyline_map(Raw_map,hyeryeong_raw,"red")
draw_Polyline_map(Raw_map,pyeongju_raw,"blue")
draw_circle_map(Raw_map,hyeryeong_raw,3,"red")
draw_circle_map(Raw_map,pyeongju_raw,3,"blue")
Raw_map

In [17]:
FILTER_CONSTANT = 10 #(min)
THRESHOLD = 15 #(m)

def moving_avg_filter(UID, scnario):
    marker_list = extract_raw_data(UID, scnario)
    avg_point_list = []
    for idx in range(len(marker_list)):
        x = marker_list[idx][1]
        y = marker_list[idx][2]
        avg_x = 0
        avg_y = 0
        for avgIdx in range(idx,idx + FILTER_CONSTANT):
            try:
                avg_x += float(marker_list[avgIdx][1])
                avg_y += float(marker_list[avgIdx][2])
            except IndexError:
                break
        avg_x = avg_x / FILTER_CONSTANT
        avg_y = avg_y / FILTER_CONSTANT
        point = [marker_list[idx][0], avg_x, avg_y]
        avg_point_list.append(point)
    return avg_point_list

kyounghwan_avg = moving_avg_filter("3963650B38CAF112", 1)
hyeryeong_avg = moving_avg_filter('4CCF4C0B0F953026', 1)
pyeongju_avg = moving_avg_filter('8A57AE95DD4629A7', 1)
joongho_avg = moving_avg_filter('9E7F4CB4F4A6A6D4', 1)
yujin_avg = moving_avg_filter('F9888CC2D3053565', 1)
hyeryeong_avg = hyeryeong_avg[:-1]
pyeongju_avg = pyeongju_avg[:-1]

def speed_vector(UID, scnario):
    moving_filter_list = moving_avg_filter(UID, scnario)
    speed_list = []
    for idx in range(len(moving_filter_list)-1):
        speed_x = moving_filter_list[idx + 1][1] - moving_filter_list[idx][1]
        speed_y = moving_filter_list[idx + 1][2] - moving_filter_list[idx][2]
        speed_point = [moving_filter_list[idx][0], speed_x, speed_y]
        speed_list.append(speed_point)
    return speed_list
        
kyounghwan_speed = speed_vector("3963650B38CAF112", 1)
hyeryeong_speed = speed_vector('4CCF4C0B0F953026', 1)
pyeongju_speed = speed_vector('8A57AE95DD4629A7', 1)
joongho_speed = speed_vector('9E7F4CB4F4A6A6D4', 1)
yujin_speed = speed_vector('F9888CC2D3053565', 1)

def trajectory_similarity(Patient_UID, Tester_UID, scnario):
    Patient_raw = extract_raw_data(Patient_UID, scnario)
    Patient_speed = speed_vector(Patient_UID, scnario)
    Tester_raw = extract_raw_data(Tester_UID, scnario)
    Tester_speed = speed_vector(Tester_UID, scnario)
    
    time_stamp = extractTime(Patient_UID, scnario)
    contact_df = pd.DataFrame(columns=["TIME", "Distance", "tractory_sim"])
    contact_df["TIME"] = time_stamp
    contact_df.set_index("TIME", inplace = True)
    for time in time_stamp:
        distance = 0
        for p_idx in range(len(Patient_raw)):
            if Patient_raw[p_idx][0] == time:
                for t_idx in range(len(Tester_raw)):
                    if Tester_raw[t_idx][0] == time:
                        distance = haversine(Patient_raw[p_idx][1:3],Tester_raw[t_idx][1:3], unit = 'm')
                        contact_df.loc[time,"Distance"] = distance
        if distance <= THRESHOLD:
            for p_idx in range(len(Patient_speed)):
                if Patient_speed[p_idx][0] == time:
                    for t_idx in range(len(Tester_speed)):
                        if Tester_speed[t_idx][0] == time:
                            cosine_sim = dot(Patient_speed[p_idx][1:3], Tester_speed[t_idx][1:3]) / (norm(Patient_speed[p_idx][1:3]) * norm(Tester_speed[t_idx][1:3]))
                            contact_df.loc[time,"tractory_sim"] = cosine_sim
        else : 
            contact_df.loc[time,"tractory_sim"] = 0
    return contact_df

KH_JH_contact = trajectory_similarity("4CCF4C0B0F953026",'8A57AE95DD4629A7', 1)
KH_JH_contact


,Distance,tractory_sim
TIME,,
2022-08-26 14:32:00,24.189061,0
2022-08-26 14:33:00,9.274413,0.999213
2022-08-26 14:34:00,15.19715,0
2022-08-26 14:35:00,16.578924,0
2022-08-26 14:36:00,16.004396,0
2022-08-26 14:37:00,15.584178,0
2022-08-26 14:38:00,41.149904,0
2022-08-26 14:39:00,11.795063,0.99964
2022-08-26 14:40:00,7.218425,0.99974


In [18]:
draw_Polyline_map(Filter_map,hyeryeong_avg,"red")
draw_Polyline_map(Filter_map,pyeongju_avg,"blue")
draw_circle_map(Filter_map,hyeryeong_avg,3,"red")
draw_circle_map(Filter_map,pyeongju_avg,3,"blue")
Filter_map